In [1]:
pip install requests pandas scikit-learn


In [4]:
API_KEY = "7fb00013e6a26db7dadb8a42a5890a50"


In [5]:
import requests
import pandas as pd

movies = []

for page in range(1, 6):  # fetch ~100 movies
    url = (
        f"https://api.themoviedb.org/3/discover/movie"
        f"?api_key={API_KEY}"
        f"&with_original_language=te"
        f"&region=IN"
        f"&page={page}"
    )

    response = requests.get(url)
    data = response.json()

    for movie in data['results']:
        if movie['overview']:
            movies.append({
                "title": movie["title"],
                "overview": movie["overview"],
                "release_date": movie["release_date"],
                "rating": movie["vote_average"]
            })

df = pd.DataFrame(movies)
df.head()


,title,overview,release_date,rating
0,Laila,"Sonu Model, a renowned beautician from the old...",2025-02-14,5.200
1,Shivam,The story of a boy (Ram) who wants to make his...,2015-10-02,4.750
2,RRR,A fictional history of two legendary revolutio...,2022-03-25,7.746
3,Devara: Part 1,"Devara, a fearless man from a coastal region, ...",2024-09-27,6.936
4,Hi Nanna,"Six-year-old Mahi, a spirited child with cysti...",2023-12-07,8.000


In [6]:
df.to_csv("telugu_movies.csv", index=False)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english')
vectors = tfidf.fit_transform(df['overview'])

similarity = cosine_similarity(vectors)

def recommend(movie):
    if movie not in df['title'].values:
        print(f"Movie '{movie}' not found in the database. Please try another movie.\n")
        return

    index = df[df['title'] == movie].index[0]
    scores = list(enumerate(similarity[index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:6]

    print(f"\nMovies similar to {movie}:\n")
    for i in scores:
        print(df.iloc[i[0]].title)

In [15]:
recommend("Hi Nanna")


Movies similar to Hi Nanna:

Janatha Garage
8 Vasantalu
Simhadri
Pushpa 2 - The Rule
Bhagavanth Kesari


In [12]:
from google.colab import files
files.download("telugu_movies.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import requests

# Fetch genre list
genre_url = f"https://api.themoviedb.org/3/genre/movie/list?api_key={API_KEY}&language=en-US"
genre_data = requests.get(genre_url).json()

genre_map = {g['id']: g['name'] for g in genre_data['genres']}
genre_map


{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [17]:
import pandas as pd

movies = []

for page in range(1, 6):  # increase pages if needed
    discover_url = (
        f"https://api.themoviedb.org/3/discover/movie"
        f"?api_key={API_KEY}"
        f"&with_original_language=te"
        f"&region=IN"
        f"&page={page}"
    )

    data = requests.get(discover_url).json()

    for movie in data['results']:
        movie_id = movie['id']

        # Get cast
        credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}"
        credits = requests.get(credits_url).json()

        cast_names = []
        for c in credits.get('cast', [])[:5]:  # top 5 actors
            cast_names.append(c['name'])

        # Get genres
        genre_names = [genre_map[g] for g in movie.get('genre_ids', [])]

        if movie['overview']:
            movies.append({
                "title": movie['title'],
                "overview": movie['overview'],
                "genres": " ".join(genre_names),
                "cast": " ".join(cast_names)
            })

df = pd.DataFrame(movies)
df.head()


,title,overview,genres,cast
0,Laila,"Sonu Model, a renowned beautician from the old...",Comedy Romance,Vishwak Sen Akanksha Sharma Babloo Prithiveera...
1,Shivam,The story of a boy (Ram) who wants to make his...,Action Comedy Romance,Ram Pothineni Raashii Khanna Abhimanyu Singh B...
2,RRR,A fictional history of two legendary revolutio...,Action Adventure Drama,N.T. Rama Rao Jr. Ram Charan Olivia Morris Ray...
3,Devara: Part 1,"Devara, a fearless man from a coastal region, ...",Action Drama,N.T. Rama Rao Jr. Saif Ali Khan Prakash Raj Sr...
4,Hi Nanna,"Six-year-old Mahi, a spirited child with cysti...",Drama Family Romance,Nani Mrunal Thakur Baby Kiara Khanna Jayaram S...


In [18]:
df['tags'] = df['overview'] + " " + df['genres'] + " " + df['cast']


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
vectors = tfidf.fit_transform(df['tags'])

similarity = cosine_similarity(vectors)

def recommend(movie):
    if movie not in df['title'].values:
        print(f"Movie '{movie}' not found in the database. Please try another movie.\n")
        return

    index = df[df['title'] == movie].index[0]
    scores = list(enumerate(similarity[index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:6]

    print(f"\nMovies similar to {movie}:\n")
    for i in scores:
        print(df.iloc[i[0]].title)

In [22]:
recommend("Hi Nanna")


Movies similar to Hi Nanna:

Janatha Garage
Sita Ramam
Simhadri
8 Vasantalu
Bhagavanth Kesari
